In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import col, to_date, when, concat, lit
from pyspark.sql.types import TimestampType
import matplotlib.pyplot as plt
import numpy as np

spark = SparkSession.builder \
    .appName("green preprocessing") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .config("spark.sql.parquet.cacheMetadata", "true") \
    .config("spark.sql.session.timeZone", "Etc/UTC") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/09/14 02:06:44 WARN Utils: Your hostname, NivethanAsus resolves to a loopback address: 127.0.1.1; using 172.20.150.32 instead (on interface eth0)
23/09/14 02:06:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/14 02:06:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/14 02:06:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/09/14 02:06:47 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/09/14 02:06:47 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
path1 = "../../../data/curated/all_data_combined.parquet"
df = spark.read.parquet(path1)
df.show(5)
df.count()
df.printSchema()


+-----------------+------------+--------------+-------+-----------+-------------+--------------+---------------+-----------------------------+----------------------------+--------------------+--------------------+----------------------+--------------------+----------------------------+--------------------------------------+----------------------------+-----------------------------------------------+-----------------------------------------------+------------------------------------+-------------------------------------------+------------------------------------------+-------------------------------------------+------------------------------------+
|consumer_postcode|merchant_abn|order_datetime|user_id|consumer_id|consumer_name|consumer_state|consumer_gender|transaction_dollar_value_$AUD|consumer_fraud_probability_%|       merchant_name|merchant_description|merchant_revenue_level|merchant_take_rate_%|merchant_fraud_probability_%|consumer_postcode_estimated_population|consumer_postcode_m